In [2]:
import requests
import boto3
import json
import os
import sys
import argparse
import importlib
import transformers
import torch
import pathlib
import awswrangler as wr
from IPython.display import display
from sagemaker.huggingface.processing import HuggingFaceProcessor
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.processing import FrameworkProcessor
from sagemaker.sklearn.estimator import SKLearn
from sagemaker.workflow.steps import ProcessingStep
from sagemaker.workflow.pipeline_context import PipelineSession
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.session import get_execution_role


# Adding ../01_modules or ./01_modules to the system path so that we can load modules from 
# there as well
if '__file__' in globals():
    script_dir = pathlib.Path(__file__).parent.resolve()
else:
    script_dir = pathlib.Path().absolute()
modules_path_in_dev = os.path.abspath(os.path.join(script_dir, '..', '01_modules'))
modules_path_in_prod = os.path.abspath(os.path.join(script_dir, '01_modules'))
if os.path.exists(modules_path_in_dev):
    sys.path.append(modules_path_in_dev)
if os.path.exists(modules_path_in_prod):
    sys.path.append(modules_path_in_prod)


# # Jupyter only reads a local module the first time after 
# # kernel start. Re-running a cell with 
# # "from mymodulename import *" would not change
# # anything, even if the imported module has since changed.
# # As a workaround, we need to directly load the module, 
# # use importlib.reload to reload it and then import * 
import utils
_ = importlib.reload(utils)
import config
_ = importlib.reload(config) 


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
config.py loaded: v0.1
utils.py loaded: v0.2.12
utils.py loaded: v0.2.12
config.py loaded: v0.1


In [3]:
utils.pd_set_options(cols=500)

wr.athena.read_sql_query("""
SELECT
    *
 FROM 
     "01_raw".semanticscholar_s2orc_v2 -- v2 has an _ prefix, it is removed from dowstream models
 LIMIT 5
 """, '01_raw')

,corpusid,openaccessinfo,title,authors,body,bibliography
0,39964499,"{'disclaimer': 'This content is derived from https://arxiv.org/abs/1208.0130. ', 'externalids': {'medline': '23214774v1', 'mag': '2081684912', 'acl': None, 'doi': '10.1103/PhysRevE.86.051302', 'medrxiv': None, 'pubmedcentral': None, 'arxiv': '1208.0130'}, 'license': None, 'url': 'https://arxiv.org/abs/1208.0130', 'status': 'GREEN'}",Origin of rebounds with a restitution coefficient larger than unity in nanocluster collisions.,"[H. Kuninaka, H. Hayakawa]","{'text': ' I. INTRODUCTION Nanoclusters are technologically important for the construction of nanodevices. Because the size of nanoclusters is mesoscopic, thermodynamic properties of such materials are still not well understood [1], though the methods to make nanoclusters such as adiabatic expansion through a nozzle and a laser ablation technique are well established. [2] Dynamics of nanoclusters are extensively investigated from both scientific and technological interest. There are many nu...","{'text': 'AcknowledgmentsWe would like to thank N. V. Brilliantov, T. Kawasaki, S. Takesue, R. Murakami, and K. Saitoh for their valuable comments.Also, HK would like to thank Y. Wang and W. Lechner who gave him many advises for calculation of order parameters and how to use their calculation codes.Parts of numerical computation in this work were carried out at Yukawa Institute Computer Facility.This work was supported by the Grant-in-Aid for the Global COE Program ""The Next Generation of Ph..."
1,234595201,"{'disclaimer': 'This content is derived from https://pmc.ncbi.nlm.nih.gov/articles/PMC8100426. Its open-access license is CCBYNCSA.', 'externalids': {'medline': '33995511v1', 'mag': None, 'acl': None, 'doi': '10.11909/j.issn.1671-5411.2021.04.010', 'medrxiv': None, 'pubmedcentral': '8100426', 'arxiv': None}, 'license': 'CCBYNCSA', 'url': 'https://pmc.ncbi.nlm.nih.gov/articles/PMC8100426', 'status': None}",Prevention of self-harm through early detection of depression among the elderly with permanent pacemaker: a case report,"[Yu-Wei Chang, Ju-Yi Chen]","{'text': ' gnosis of sick sinus syndrome. Multiple somatic complaints, especially chronic back pain, had impacted the patient's sleep quality and daily life. The patient also had a diagnosis of general anxiety disorder, and she had been regularly visiting a psychiatric clinic for insomnia over the two years preceding this event. The patient occasionally expressed suicidal ideation but her family did not pay much attention to her suicidal verbalizations. At about 2:00 a.m., during the night...","{'text': 'ACKNOWLEDGMENTSThis study was supported by the Ministry of Science and Technology of Taiwan, China (MOST 108-2218-E-006-019 & MOST 109-2218-E-006-024).All authors had no conflicts of interest to disclose. Depression in older adults. A Fiske, J L Wetherell, M Gatz, Annu Rev Clin Psychol. 52009 . F M Kusumoto, M H Schoenfeld, C Barrett, 2018 ACC/AHA/HRS guideline on the evaluation and management of patients with bradycardia and cardiac conduction delay: a report of the American Col..."
2,14415362,"{'disclaimer': 'This content is derived from https://arxiv.org/abs/0705.0584. ', 'externalids': {'medline': None, 'mag': '2031777933', 'acl': None, 'doi': '10.1007/S00605-008-0535-3', 'medrxiv': None, 'pubmedcentral': None, 'arxiv': '0705.0584'}, 'license': None, 'url': 'https://arxiv.org/abs/0705.0584', 'status': 'CLOSED'}",Multidimensional continued fractions and a Minkowski function,[Giovanni Panti],"{'text': ' Preliminaries The nth order Farey set F n in the real unit interval [0, 1] is defined by recursion: one starts with F 0 = {0/1, 1/1} and obtains F n by adding to F n−1 all the Farey sums v 1 ⊕ v 2 = (a 1 + a 2 )/(b 1 + b 2 ) of two consecutive elements v i = a i /b i of F n−1 . The union of all the F n 's is the set of all rational numbers in [0, 1]. Analogously, by starting with B 0 = F 0 and replacing the Farey sum with the barycentric sum v 1 v 2 = (v 1 + v 2 

In [13]:
wr.athena.read_sql_query("""
SELECT
    corpusid AS id_semanticscholar,
    JSON_EXTRACT_SCALAR(CAST(openaccessinfo AS JSON), '$.externalids.mag') AS id_mag,
    JSON_EXTRACT_SCALAR(CAST(openaccessinfo AS JSON), '$.externalids.doi') AS id_doi,
    JSON_EXTRACT_SCALAR(CAST(openaccessinfo AS JSON), '$.externalids.arxiv') AS id_arxiv,
    title,
    JSON_EXTRACT_SCALAR(CAST(openaccessinfo AS JSON), '$.license') AS license,
    JSON_EXTRACT_SCALAR(CAST(openaccessinfo AS JSON), '$.url') AS source_url,
    JSON_EXTRACT_SCALAR(CAST(openaccessinfo AS JSON), '$.status') AS openaccess_status,
    JSON_EXTRACT_SCALAR(CAST(body AS JSON), '$.text') AS content_text,
    JSON_EXTRACT_SCALAR(CAST(body AS JSON), '$.annotations.paragraph') AS annotations_paragraph,
    JSON_EXTRACT_SCALAR(CAST(body AS JSON), '$.annotations.section_header') AS annotations_section_header
    --'x' AS "x"
 FROM 
     "01_raw".semanticscholar_s2orc_v2
 WHERE
     TRUE
 LIMIT 10
 """, '01_raw')

,id_semanticscholar,id_mag,id_doi,id_arxiv,title,license,source_url,openaccess_status,content_text,annotations_paragraph,annotations_section_header
0,234869482,3155033926,10.5772/INTECHOPEN.97307,<NA>,Thoracoabdominal Compartment Syndrome,CCBY,https://doi.org/10.5772/INTECHOPEN.97307,HYBRID,"\nIntroduction\n\nThe diaphragm is formed from a number of composite origins in the embryo. The most important is the ""septum transversum"", which is a thick mass of cranial mesenchyme that gives rise to parts of the thoracic diaphragm. Without dwelling into more details, the septum transversum merges with mesoderm surrounding the esophagus, the growing pleura and peritoneum ('pleuroperitoneal folds') and the growing muscles of the abdominal wall. The septum transversum gives rise to the cent...","[{""attributes"":null,""end"":612,""start"":15},{""attributes"":null,""end"":730,""start"":614},{""attributes"":null,""end"":999,""start"":732},{""attributes"":null,""end"":1550,""start"":1001},{""attributes"":null,""end"":1777,""start"":1552},{""attributes"":null,""end"":2191,""start"":1779},{""attributes"":null,""end"":2552,""start"":2193},{""attributes"":null,""end"":2851,""start"":2554},{""attributes"":null,""end"":3350,""start"":2880},{""attributes"":null,""end"":3706,""start"":3352},{""attributes"":null,""end"":3930,""start"":3708},{""attributes"":null...","[{""attributes"":{""n"":""1.""},""end"":13,""start"":1},{""attributes"":{""n"":""2.""},""end"":2878,""start"":2853},{""attributes"":{""n"":""3.""},""end"":5257,""start"":5201},{""attributes"":{""n"":""3.1""},""end"":5279,""start"":5259},{""attributes"":{""n"":""3.2""},""end"":6340,""start"":6286},{""attributes"":{""n"":""3.3""},""end"":9066,""start"":9029},{""attributes"":{""n"":""4.""},""end"":10473,""start"":10463}]"
1,270297560,<NA>,10.3390/dj12060170,<NA>,Periodontal Regeneration of Vital Poor Prognosis Teeth with Attachment Loss Involving the Root Apex: Two Cases with up to 5 Years Follow-Up,CCBY,https://pmc.ncbi.nlm.nih.gov/articles/PMC11202695,GOLD,"\nIntroduction\n\nPeriodontal regeneration is the ultimate goal of periodontal therapy and is defined as the ""restoration of lost or diminished periodontal tissues including cementum, periodontal ligament, and alveolar bone"" [1].Guided tissue regeneration (GTR), on the other hand, refers to a surgical procedure seeking to obtain the objectives of periodontal regeneration through utilising barrier devices or membranes, to exclude epithelial cells and provide space maintenance [1].The clinical...","[{""attributes"":null,""end"":1005,""start"":15},{""attributes"":null,""end"":3444,""start"":1007},{""attributes"":null,""end"":4719,""start"":3469},{""attributes"":null,""end"":5712,""start"":4721},{""attributes"":null,""end"":6359,""start"":5722},{""attributes"":null,""end"":9027,""start"":6369},{""attributes"":null,""end"":9648,""start"":9037},{""attributes"":null,""end"":10673,""start"":9650},{""attributes"":null,""end"":12286,""start"":10683},{""attributes"":null,""end"":12578,""start"":12297},{""attributes"":null,""end"":13477,""start"":12589},{""attr...","[{""attributes"":{""n"":""1.""},""end"":13,""start"":1},{""attributes"":{""n"":""2.""},""end"":3467,""start"":3446},{""attributes"":{""n"":""2.1.""},""end"":5720,""start"":5714},{""attributes"":{""n"":""2.1.""},""end"":6367,""start"":6361},{""attributes"":{""n"":""2.2.""},""end"":9035,""start"":9029},{""attributes"":{""n"":""2.2.""},""end"":10681,""start"":10675},{""attributes"":{""n"":""3.""},""end"":12295,""start"":12288},{""attributes"":{""n"":""3.""},""end"":12587,""start"":12580},{""attributes"":{""n"":""4.""},""end"":13489,""start"":13479},{""attributes"":{""n"":""4.""},""end"":142..."
2,251953573,<NA>,10.1103/PhysRevD.108.094029,2208.14827,Entanglement renormalization of the class of continuous matrix product states,CCBY,https://arxiv.org/abs/2208.14827,HYBRID,"\nTensor Network states are the entanglement-based ansatz that has arisen in recent years based on the renormalizati

In [9]:
utils.pd_set_options()
wr.athena.read_sql_query("""
WITH
raw_semanticscholar_s2orcv2 AS 
(
    SELECT * FROM "01_raw".semanticscholar_s2orc_v2
),
base_semanticscholar_s2orcv2_step01 AS 
(
    SELECT
        corpusid AS id_semanticscholar,
        JSON_EXTRACT_SCALAR(CAST(openaccessinfo AS JSON), '$.externalids.mag') AS id_mag,
        JSON_EXTRACT_SCALAR(CAST(openaccessinfo AS JSON), '$.externalids.doi') AS id_doi,
        JSON_EXTRACT_SCALAR(CAST(openaccessinfo AS JSON), '$.externalids.arxiv') AS id_arxiv,
        title,
        JSON_EXTRACT_SCALAR(CAST(openaccessinfo AS JSON), '$.license') AS original_license,
        JSON_EXTRACT_SCALAR(CAST(openaccessinfo AS JSON), '$.url') AS source_url,
        JSON_EXTRACT_SCALAR(CAST(openaccessinfo AS JSON), '$.status') AS openaccess_status,
        JSON_EXTRACT_SCALAR(CAST(body AS JSON), '$.text') AS content_text,
        JSON_EXTRACT_SCALAR(CAST(body AS JSON), '$.annotations.paragraph') AS annotations_paragraph,
        JSON_EXTRACT_SCALAR(CAST(body AS JSON), '$.annotations.section_header') AS annotations_section_header
     FROM 
         raw_semanticscholar_s2orcv2
),
base_semanticscholar_s2orcv2 AS (
    SELECT
        id_semanticscholar,
        id_mag,
        id_doi,
        id_arxiv,
        title,
        source_url,
        openaccess_status,
        content_text,
        annotations_paragraph,
        annotations_section_header,
        CASE
            -- These licenses might or might not be usable just because they are openly accessible, so we are assuming they are not
            WHEN 
                original_license IS NULL OR
                original_license IN(
                    'acs-specific: authorchoice/editors choice usage agreement',
                    'elsevier-specific: oa user license',
                    'other-oa',
                    'publisher-specific, author manuscript: http://academic.oup.com/journals/pages/about_us/legal/notices',
                    'publisher-specific-oa',
                    'unspecified-oa',
                    'Open Government Licence - Canada',
                    'publisher-specific, author manuscript',
                    'implied-oa',
                    'publisher-specific, author manuscript: http://rsc.li/journals-terms-of-use',
                    'publisher-specific, author manuscript: http://onlinelibrary.wiley.com/termsAndConditions',
                    'publisher-specific, author manuscript: http://onlinelibrary.wiley.com/termsAndConditions#am',
                    'publisher-specific license'
                ) 
            THEN 'unknown-reusability'
            -- A few public domain licenses were abbreviated
            WHEN original_license = 'pd' THEN 'public-domain'
            ELSE original_license
        END AS license
     FROM 
         base_semanticscholar_s2orcv2_step01
)
SELECT * FROM base_semanticscholar_s2orcv2 
LIMIT 5
""", '01_raw')

,license
0,CCBYND
1,unknown-reusability
2,CCBYNCSA
3,CCBYNC
4,public-domain
5,gpl
6,CC0
7,CCBY
8,CCBYSA
9,CCBYNCND


In [10]:
utils.pd_set_options(cols=500)

wr.athena.read_sql_query("""
SELECT
    license,
    COUNT(*) AS c
 FROM 
     "02_stg".base_semanticscholar_s2orcv2
 GROUP BY
     license
 """, '02_stg')

,license,c
0,CCBYND,18510
1,gpl,11
2,public-domain,22963
3,CCBYNC,1131261
4,CCBYNCSA,333540
5,CCBY,6487518
6,CCBYSA,127094
7,mit,5443
8,unknown-reusability,2526433
9,CCBYNCND,945585


In [10]:
utils.pd_set_options(cols=500)

wr.athena.read_sql_query("""
WITH
base_semanticscholar_s2orcv2_ AS (
    SELECT * FROM "02_stg".base_semanticscholar_s2orcv2
),
raw_semanticscholar_papers AS (
    SELECT * FROM "01_raw".semanticscholar_papers
),
raw_semanticscholar_abstracts AS (
    SELECT * FROM "01_raw".semanticscholar_abstracts
),
semanticscholar_joined AS (
    SELECT
        base_semanticscholar_s2orcv2_.id_semanticscholar,
        base_semanticscholar_s2orcv2_.id_mag,
        base_semanticscholar_s2orcv2_.id_doi,
        base_semanticscholar_s2orcv2_.id_arxiv,
        base_semanticscholar_s2orcv2_.title,
        base_semanticscholar_s2orcv2_.source_url,
        base_semanticscholar_s2orcv2_.openaccess_status,
        base_semanticscholar_s2orcv2_.content_text,
        base_semanticscholar_s2orcv2_.annotations_paragraph,
        base_semanticscholar_s2orcv2_.annotations_section_header,
        base_semanticscholar_s2orcv2_.license,

        CASE WHEN raw_semanticscholar_abstracts.corpusid IS NULL THEN 0 ELSE 1 END AS abstracts_join_worked,
        raw_semanticscholar_abstracts.abstract AS content_abstract,

        CASE WHEN raw_semanticscholar_papers.corpusid IS NULL THEN 0 ELSE 1 END AS papers_join_worked,
        raw_semanticscholar_papers.year AS publication_year,
        raw_semanticscholar_papers.publicationdate AS publication_date
    FROM
        base_semanticscholar_s2orcv2_
    LEFT JOIN
        raw_semanticscholar_papers
    ON
        base_semanticscholar_s2orcv2_.id_semanticscholar = raw_semanticscholar_papers.corpusid
    LEFT JOIN
        raw_semanticscholar_abstracts
    ON
        base_semanticscholar_s2orcv2_.id_semanticscholar = raw_semanticscholar_abstracts.corpusid
),
count_join_to_abstracts AS (
SELECT
    'abstracts' AS join_type,
    abstracts_join_worked AS join_worked,
    COUNT(*) AS c
FROM
    semanticscholar_joined
GROUP BY
    abstracts_join_worked
),
count_join_to_papers AS (
SELECT
    'papers' AS join_type,
    papers_join_worked AS join_worked,
    COUNT(*) AS c
FROM
    semanticscholar_joined
GROUP BY
    papers_join_worked
),
count_s2orcv2 AS (
SELECT
    'original' AS join_type,
    CAST(NULL AS INT) AS join_worked,
    COUNT(*) AS c
FROM
    base_semanticscholar_s2orcv2_
)
SELECT * FROM count_join_to_abstracts UNION ALL
SELECT * FROM count_join_to_papers UNION ALL
SELECT * FROM count_s2orcv2
 """, '02_stg')

,join_type,join_worked,c
0,original,<NA>,11609787
1,abstracts,1,10910456
2,abstracts,0,699331
3,papers,1,11609787


In [19]:
utils.pd_set_options(cols=500)

wr.athena.read_sql_query("""
WITH
base_semanticscholar_s2orcv2_ AS (
SELECT
    *
FROM 
     "02_stg".base_semanticscholar_s2orcv2
),
flags AS (
SELECT 
    CASE WHEN id_semanticscholar IS NULL THEN 0 ELSE 1 END AS has_id_semanticscholar,
    CASE WHEN id_mag IS NULL THEN 0 ELSE 1 END AS has_id_mag,
    CASE WHEN id_doi IS NULL THEN 0 ELSE 1 END AS has_id_doi,
    CASE WHEN id_arxiv IS NULL THEN 0 ELSE 1 END AS has_id_arxiv--,
    --*
FROM
    base_semanticscholar_s2orcv2_
)
SELECT has_id_arxiv, COUNT(*)/1000000.0 AS c FROM flags GROUP BY has_id_arxiv
 """, '02_stg')

,has_id_arxiv,c
0,1,2.600510
1,0,9.009277


In [8]:
utils.pd_set_options(cols=500)

wr.athena.read_sql_query("""
WITH
base_semanticscholar_s2orcv2_ AS (
    SELECT * FROM "02_stg".base_semanticscholar_s2orcv2
),
base_arxiv_metadata_ AS (
    SELECT * FROM "02_stg".base_arxiv_metadata
),
join_on_arxiv_id AS (
    SELECT
        base_semanticscholar_s2orcv2_.id_semanticscholar AS semanticscholar_id_semanticscholar,
        base_semanticscholar_s2orcv2_.id_arxiv AS semanticscholar_id_arxiv,
        base_semanticscholar_s2orcv2_.id_doi AS semanticscholar_id_doi,
        base_arxiv_metadata_.id_arxiv AS arxiv_id_arxiv,
        base_arxiv_metadata_.id_doi AS arxiv_id_doi,
        CASE WHEN base_arxiv_metadata_.id_arxiv IS NULL THEN 0 ELSE 1 END AS join_worked
    FROM
        base_semanticscholar_s2orcv2_
    LEFT JOIN
        base_arxiv_metadata_
    ON
        base_semanticscholar_s2orcv2_.id_arxiv = base_arxiv_metadata_.id_arxiv
),
count_join_on_arxiv_id AS (
SELECT
    'arxiv' AS join_type,
    COUNT(*) AS c
FROM
    join_on_arxiv_id
WHERE
    join_worked = 1
GROUP BY
    join_worked
),
join_on_doi_id AS (
    SELECT
        base_semanticscholar_s2orcv2_.id_semanticscholar AS semanticscholar_id_semanticscholar,
        base_semanticscholar_s2orcv2_.id_arxiv AS semanticscholar_id_arxiv,
        base_semanticscholar_s2orcv2_.id_doi AS semanticscholar_id_doi,
        base_arxiv_metadata_.id_arxiv AS arxiv_id_arxiv,
        base_arxiv_metadata_.id_doi AS arxiv_id_doi,
        CASE WHEN base_arxiv_metadata_.id_doi IS NULL THEN 0 ELSE 1 END AS join_worked
    FROM
        base_semanticscholar_s2orcv2_
    LEFT JOIN
        base_arxiv_metadata_
    ON
        base_semanticscholar_s2orcv2_.id_doi = base_arxiv_metadata_.id_doi
),
count_join_on_doi_id AS (
SELECT
    'doi' AS join_type,
    COUNT(*) AS c
FROM
    join_on_doi_id
WHERE
    join_worked = 1
GROUP BY
    join_worked
),
count_arxiv AS (
SELECT
    'original' AS join_type,
    COUNT(*) AS c
FROM
    base_arxiv_metadata_
)
SELECT * FROM count_join_on_arxiv_id UNION ALL
SELECT * FROM count_join_on_doi_id UNION ALL
SELECT * FROM count_arxiv
 """, '02_stg')

,join_type,c
0,original,2816721
1,arxiv,2600510
2,doi,1166552


In [12]:
utils.pd_set_options(cols=500)

wr.athena.read_sql_query("""
WITH
stg_semanticscholar_combined_works_ AS (
    SELECT * FROM "02_stg".stg_semanticscholar_combined_works
),
base_arxiv_metadata_ AS (
    SELECT * FROM "02_stg".base_arxiv_metadata
),
stg_unified_works_01_joined_to_arxiv_step01 AS (
    SELECT
        stg_semanticscholar_combined_works_.id_semanticscholar,
        stg_semanticscholar_combined_works_.id_mag,
        stg_semanticscholar_combined_works_.id_doi,
        stg_semanticscholar_combined_works_.id_arxiv,
        stg_semanticscholar_combined_works_.title,
        stg_semanticscholar_combined_works_.source_url,
        stg_semanticscholar_combined_works_.openaccess_status,
        stg_semanticscholar_combined_works_.content_text,
        stg_semanticscholar_combined_works_.annotations_paragraph,
        stg_semanticscholar_combined_works_.annotations_section_header,
        stg_semanticscholar_combined_works_.content_abstract,
        stg_semanticscholar_combined_works_.publication_year,
        stg_semanticscholar_combined_works_.publication_date,
        --stg_semanticscholar_combined_works_.license AS original_semanticscholar_license,
        --base_arxiv_metadata_.license AS arxiv_license,
        COALESCE(
            stg_semanticscholar_combined_works_.license,
            base_arxiv_metadata_.license,
            'unknown-reusability' -- If there's no info on it on either S2 or Arxiv, we consider it not usable 
        ) AS license
    FROM
        stg_semanticscholar_combined_works_
    LEFT JOIN
        base_arxiv_metadata_
    ON
        stg_semanticscholar_combined_works_.id_arxiv = base_arxiv_metadata_.id_arxiv
),
stg_unified_works_01_joined_to_arxiv AS (
SELECT
    *,
    CASE
        WHEN license IN ('public-domain', 'mit', 'gpl', 'CC0', 'CCBY', 'CCBYNC') THEN 1
        ELSE 0
    END AS license_allows_derivative_reuse
FROM
    stg_unified_works_01_joined_to_arxiv_step01
)
SELECT license_allows_derivative_reuse, license, COUNT(*)/1000000.0 AS c FROM stg_unified_works_01_joined_to_arxiv GROUP BY license_allows_derivative_reuse, license ORDER BY license_allows_derivative_reuse, c
 """, '02_stg')

,license_allows_derivative_reuse,license,c
0,0,CCBYND,0.018510
1,0,CCBYSA,0.144872
2,0,CCBYNCSA,0.378748
3,0,unknown-reusability,0.656543
4,0,CCBYNCND,0.997363
5,0,ArXiv nonexclusive-distrib,1.420597
6,1,gpl,0.000011
7,1,mit,0.005443
8,1,public-domain,0.025038
9,1,CC0,0.025536


In [ ]:
utils.pd_set_options(cols=500)

wr.athena.read_sql_query("""
WITH
stg_unified_works_01_joined_to_arxiv_ AS (
    SELECT * FROM "02_stg".stg_unified_works_01_joined_to_arxiv
),
base_openalex_metadata_ AS (
    SELECT * FROM "02_stg".base_openalex_metadata
),
stg_unified_works_02_joined_to_openalex_step01 AS (
    SELECT
        stg_semanticscholar_combined_works_.id_semanticscholar,
        stg_semanticscholar_combined_works_.id_mag,
        stg_semanticscholar_combined_works_.id_doi,
        stg_semanticscholar_combined_works_.id_arxiv,
        stg_semanticscholar_combined_works_.title,
        stg_semanticscholar_combined_works_.source_url,
        stg_semanticscholar_combined_works_.openaccess_status,
        stg_semanticscholar_combined_works_.content_text,
        stg_semanticscholar_combined_works_.annotations_paragraph,
        stg_semanticscholar_combined_works_.annotations_section_header,
        stg_semanticscholar_combined_works_.content_abstract,
        stg_semanticscholar_combined_works_.publication_year,
        stg_semanticscholar_combined_works_.publication_date,
        --stg_semanticscholar_combined_works_.license AS original_semanticscholar_license,
        --base_arxiv_metadata_.license AS arxiv_license,
        COALESCE(
            stg_semanticscholar_combined_works_.license,
            base_arxiv_metadata_.license,
            'unknown-reusability' -- If there's no info on it on either S2 or Arxiv, we consider it not usable 
        ) AS license
    FROM
        stg_semanticscholar_combined_works_
    LEFT JOIN
        base_arxiv_metadata_
    ON
        stg_semanticscholar_combined_works_.id_arxiv = base_arxiv_metadata_.id_arxiv
),
stg_unified_works_01_joined_to_arxiv AS (
SELECT
    *,
    CASE
        WHEN license IN ('public-domain', 'mit', 'gpl', 'CC0', 'CCBY', 'CCBYNC') THEN 1
        ELSE 0
    END AS license_allows_derivative_reuse
FROM
    stg_unified_works_01_joined_to_arxiv_step01
)
SELECT license_allows_derivative_reuse, license, COUNT(*)/1000000.0 AS c FROM stg_unified_works_01_joined_to_arxiv GROUP BY license_allows_derivative_reuse, license ORDER BY license_allows_derivative_reuse, c
 """, '02_stg')

In [13]:
utils.pd_set_options(cols=500)

wr.athena.read_sql_query("""
WITH
base_semanticscholar_s2orcv2_ AS (
    SELECT * FROM "02_stg".base_semanticscholar_s2orcv2
),
base_openalex_works_reduced_ AS (
SELECT * FROM "02_stg"."base_openalex_works_reduced"
),
join_on_mag_openalex_id AS (
    SELECT
        base_semanticscholar_s2orcv2_.id_semanticscholar AS semanticscholar_id_semanticscholar,
        base_semanticscholar_s2orcv2_.id_mag AS semanticscholar_id_mag,
        base_semanticscholar_s2orcv2_.id_doi AS semanticscholar_id_doi,
        base_openalex_works_reduced_.id_openalex AS openalex_id_openalex,
        base_openalex_works_reduced_.id_doi AS openalex_id_doi,
        CASE WHEN base_openalex_works_reduced_.id_openalex IS NULL THEN 0 ELSE 1 END AS join_worked
    FROM
        base_semanticscholar_s2orcv2_
    LEFT JOIN
        base_openalex_works_reduced_
    ON
        base_semanticscholar_s2orcv2_.id_mag = base_openalex_works_reduced_.id_openalex
),
count_join_on_mag_openalex_id AS (
SELECT
    'mag_openalex' AS join_type,
    COUNT(*) AS c
FROM
    join_on_mag_openalex_id
WHERE
    join_worked = 1
GROUP BY
    join_worked
),
join_on_doi_id AS (
    SELECT
        base_semanticscholar_s2orcv2_.id_semanticscholar AS semanticscholar_id_semanticscholar,
        base_semanticscholar_s2orcv2_.id_mag AS semanticscholar_id_mag,
        base_semanticscholar_s2orcv2_.id_doi AS semanticscholar_id_doi,
        base_openalex_works_reduced_.id_openalex AS openalex_id_openalex,
        base_openalex_works_reduced_.id_doi AS openalex_id_doi,
        CASE WHEN base_openalex_works_reduced_.id_doi IS NULL THEN 0 ELSE 1 END AS join_worked
    FROM
        base_semanticscholar_s2orcv2_
    LEFT JOIN
        base_openalex_works_reduced_
    ON
        base_semanticscholar_s2orcv2_.id_doi = base_openalex_works_reduced_.id_doi
),
count_join_on_doi_id AS (
SELECT
    'doi' AS join_type,
    COUNT(*) AS c
FROM
    join_on_doi_id
WHERE
    join_worked = 1
GROUP BY
    join_worked
),
count_openalex AS (
SELECT
    'original' AS join_type,
    COUNT(*) AS c
FROM
    base_openalex_works_reduced_
)
SELECT * FROM count_join_on_mag_openalex_id UNION ALL
SELECT * FROM count_join_on_doi_id UNION ALL
SELECT * FROM count_openalex
 """, '02_stg')

,join_type,c
0,original,270051911
1,mag_openalex,6122684
2,doi,8196311


In [18]:
utils.pd_set_options(cols=500)

wr.athena.read_sql_query("""
WITH
base_semanticscholar_s2orcv2_ AS (
    SELECT * FROM "02_stg".base_semanticscholar_s2orcv2
),
base_openalex_works_reduced_ AS (
SELECT * FROM "02_stg"."base_openalex_works_reduced"
),
join_on_mag_openalex_id AS (
    SELECT
        base_semanticscholar_s2orcv2_.title AS semanticscholar_title, 
        base_semanticscholar_s2orcv2_.id_semanticscholar AS semanticscholar_id_semanticscholar,
        base_semanticscholar_s2orcv2_.id_mag AS semanticscholar_id_mag,
        base_semanticscholar_s2orcv2_.id_doi AS semanticscholar_id_doi,
        base_openalex_works_reduced_.title AS openalex_title,
        base_openalex_works_reduced_.id_openalex AS openalex_id_openalex,
        base_openalex_works_reduced_.id_doi AS openalex_id_doi,
        CASE WHEN base_openalex_works_reduced_.id_openalex IS NULL THEN 0 ELSE 1 END AS join_worked
    FROM
        base_semanticscholar_s2orcv2_
    LEFT JOIN
        base_openalex_works_reduced_
    ON
        base_semanticscholar_s2orcv2_.id_mag = base_openalex_works_reduced_.id_openalex
),
join_on_doi_id AS (
    SELECT
        base_semanticscholar_s2orcv2_.title AS semanticscholar_title, 
        base_semanticscholar_s2orcv2_.id_semanticscholar AS semanticscholar_id_semanticscholar,
        base_semanticscholar_s2orcv2_.id_mag AS semanticscholar_id_mag,
        base_semanticscholar_s2orcv2_.id_doi AS semanticscholar_id_doi,
        base_openalex_works_reduced_.title AS openalex_title,
        base_openalex_works_reduced_.id_openalex AS openalex_id_openalex,
        base_openalex_works_reduced_.id_doi AS openalex_id_doi,
        CASE WHEN base_openalex_works_reduced_.id_doi IS NULL THEN 0 ELSE 1 END AS join_worked
    FROM
        base_semanticscholar_s2orcv2_
    LEFT JOIN
        base_openalex_works_reduced_
    ON
        base_semanticscholar_s2orcv2_.id_doi = base_openalex_works_reduced_.id_doi
)
--SELECT * FROM join_on_doi_id WHERE semanticscholar_id_mag != openalex_id_openalex LIMIT 10
SELECT * FROM join_on_mag_openalex_id WHERE semanticscholar_id_doi != openalex_id_doi LIMIT 10
 """, '02_stg')

,semanticscholar_title,semanticscholar_id_semanticscholar,semanticscholar_id_mag,semanticscholar_id_doi,openalex_title,openalex_id_openalex,openalex_id_doi,join_worked
0,Delphinidin Inhibits HER2 and Erk1/2 Signaling and Suppresses Growth of HER2-Overexpressing and Triple Negative Breast Cancer Cell Lines,13284383,2136532468,10.4137/BCBCR.S7156,Delphinidin Inhibits HER2 and Erk1/2 Signaling and Suppresses Growth of HER2-Overexpressing and Triple Negative Breast Cancer Cell Lines,2136532468,10.4137/bcbcr.s7156,1
1,Cc Chemokine Receptor (Ccr)2 Is Required for Langerhans Cell Migration and Localization of T Helper Cell Type 1 (Th1)-Inducing Dendritic Cells,9509792,2165299403,10.1084/JEM.192.2.205,Cc Chemokine Receptor (Ccr)2 Is Required for Langerhans Cell Migration and Localization of T Helper Cell Type 1 (Th1)-Inducing Dendritic Cells,2165299403,10.1084/jem.192.2.205,1
2,Extended Conditional Independence and Applications in Causal Inference,55355687,2963542226,10.1214/16-AOS153,Extended conditional independence and applications in causal inference,2963542226,10.1214/16-aos1537,1
3,Basic rockfall simulation with consideration of vegetation and application to protection measure.,54050674,2080120296,10.5194/NHESS-9-1835-2009,Basic rockfall simulation with consideration of vegetation and application to protection measure,2080120296,10.5194/nhess-9-1835-2009,1
4,Uniconazole increases starch content in duckweed (Lemna aequinoctialis Welw.),107823484,2922116023,10.5614/3BIO.2019.1.1.1,Uniconazole increases starch content in duckweed (Lemna aequinoctialis Welw.),2922116023,10.5614/3bio.2019.1.1.1,1
5,"Position of arm and forearm, and elbow flexion during performance of the sculling technique: Technical recommendation versus actual performance",55513289,1999060572,10.1590/S1980-65742014000100005,"Position of arm and forearm, and elbow flexion during performance of the sculling technique: Technical recommendation versus actual performance",1999060572,10.1590/s1980-65742014000100005,1
6,Negative regulation of ABA signaling by WRKY33 is critical for Arabidopsis immunity towards Botrytis cinerea 2100,3013333,1958695491,10.7554/eLife.07295,Negative regulation of ABA signaling by WRKY33 is critical for Arabidopsis immunity towards Botrytis cinerea 2100,1958695491,10.7554/elife.07295,1
7,An inhibitory gate for state transition in cortex,5523673,2616328221,10.7554/eLife.26177,An inhibitory gate for state transition in cortex,2616328221,10.7554/elife.26177,1
8,Clinical Outcome of Dome Osteotomy in Cubitus Varus,39498644,2112467383,10.15406/MOJOR.2014.01.00022,Clinical Outcome of Dome Osteotomy in Cubitus Varus,2112467383,10.15406/mojor.2014.01.00022,1
9,Oxygen stable isotopes during the Last Glacial Maximum climate: perspectives from data-model (iLOVECLIM) comparison,56264029,2075607405,10.5194/CP-10-1939-2014,Oxygen stable isotopes during the Last Glacial Maximum climate: perspectives from data–model (&amp;lt;i&amp;gt;i&amp;lt;/i&amp;gt;LOVECLIM) comparison,2075607405,10.5194/cp-10-1939-2014,1
